### Run the following cell only once after kernel started

In [ ]:
import os
! rm -rf ondewo-t2s-client-python
! git clone https://github.com/ondewo/ondewo-t2s-client-python.git
! cd ondewo-t2s-client-python && git checkout tags/1.5.0 -b 1.5.0 && pip install -r requirements.txt  && pip install soundfile numpy && git status

### Make sure you are in "ondewo-t2s-client-python" folder

In [ ]:
os.getcwd()
! cd ondewo-t2s-client-python && python -m pip install .

In [ ]:
import io
from typing import Any

import IPython.display as ipd
import soundfile as sf

from ondewo.t2s import text_to_speech_pb2
from ondewo.t2s.client.client import Client
from ondewo.t2s.client.client_config import ClientConfig
from ondewo.t2s.client.services.text_to_speech import Text2Speech
from ondewo.t2s.text_to_speech_pb2 import ListT2sPipelinesRequest, Text2SpeechConfig

### Set up the parameters of the grpc server. The example below is for the case when server is running locally

In [ ]:
MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "<ADD GRPC SERVER HERE>"
GRPC_PORT: str = "<ADD GRPC PORT HERE>"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
cert="<ADD GRPC CERTIFICATE HERE>"

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=True)


Client

In [ ]:
t2s_service: Text2Speech = client.services.text_to_speech
pipelines = t2s_service.list_t2s_pipelines(request=ListT2sPipelinesRequest()).pipelines
print(pipelines)

### Select pipelines for specific language language

In [ ]:
def find_pipeline_for_language(pipelines, language):
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

In [ ]:
english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

In [ ]:
pipelines

### Make synthesize request to the server to get audio for given text

In [ ]:
request = text_to_speech_pb2.SynthesizeRequest(text='hallo das ist mein erster satz in Deutsch!', t2s_pipeline_id=german_pipeline.id, length_scale = 1.0, pcm=0, audio_format= 0)
# List pipelines based on conditions
german_pipelines = t2s_service.list_t2s_pipelines(
    request=ListT2sPipelinesRequest(languages=["de"])
).pipelines
german_pipeline: Text2SpeechConfig = german_pipelines[0]

response = t2s_service.synthesize(request=request)

print(
    f"Length of the generated audio is {response.audio_length} sec.",
    f"Generation time is {response.generation_time} sec.",
)

bio = io.BytesIO(response.audio)
audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio, )

ipd.Audio(audio[0], rate=audio[1])